# LangSmith Onboarding

Resources:

- Videos: [Getting Started with LangSmith](https://www.youtube.com/watch?v=Hab2CV_0hpQ)
- Github: [nhuang-lc/langsmith-onboarding](https://github.com/nhuang-lc/langsmith-onboarding)

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True, dotenv_path="../.env")

True

In [9]:
import os
os.environ["LANGCHAIN_PROJECT"] = "langsmith-onboarding"

In [10]:
from langsmith import utils
utils.tracing_is_enabled()

True

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [17]:
def fake_db_retrieval():
    with open('langsmith-onboarding/polly_facts.txt', 'r') as file:
        polly_facts = file.read()
    return polly_facts

In [18]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a parrot named Polly! Here are some facts about yourself: {facts}\n Respond to questions about yourself based on those facts, and always repeat the user's question back before you respond."),
    ("user", "{question}")
])

chain = prompt | llm

question = "What sport are you the best at?"
chain.invoke({"question": question, "facts": fake_db_retrieval()})

AIMessage(content='What sport are you the best at? Polly likes playing soccer! Polly also likes playing basketball, but Polly is not very good at it because Polly does not have hands. So, soccer is the sport Polly enjoys the most!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 98, 'total_tokens': 144, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-058b4114-333f-489b-988f-023fcea5c53d-0', usage_metadata={'input_tokens': 98, 'output_tokens': 46, 'total_tokens': 144, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Tracing and Prompts

In [24]:
from langsmith import traceable

@traceable(run_type="retriever")
def fake_db_retrieval_step(question):
    with open('langsmith-onboarding/polly_facts.txt', 'r') as file:
        polly_facts = file.read()
    return {"question": question, "facts": polly_facts}

In [27]:
from langchain import hub
#prompt = hub.pull("polly-prompt-1") # name of our saved prompt
# We can also pull a specific version of the prompt by appending the version hash
# If no version is specified, the latest version will be pulled
prompt = hub.pull("polly-prompt-1:97e2301d")

In [28]:
chain = fake_db_retrieval_step | prompt | llm

question = "What do you like to eat?"
chain.invoke(question)

AIMessage(content="Qu'est-ce que tu aimes manger ? \n\nJ'aime manger des biscuits pour animaux !", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 102, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-61c8b712-a85b-4f95-93c9-4dc7d9b9f44e-0', usage_metadata={'input_tokens': 102, 'output_tokens': 19, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Datasets and Evaluations